In [ ]:
!sudo apt-get install git-lfs -y

!git clone https://github.com/amazon-science/esci-data.git

In [ ]:
!ls esci-data/shopping_queries_dataset

In [ ]:
!pip install langdetect  pandarallel  chardet


In [ ]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

In [ ]:
import pandas as pd
from langdetect import detect, detect_langs
from langdetect import detect, LangDetectException

import string
import unicodedata
from unidecode import unidecode
import chardet

In [ ]:
!ls -lah esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet

Load examples, products and sources


In [ ]:
df_examples = pd.read_parquet('esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet')
df_products = pd.read_parquet('esci-data/shopping_queries_dataset/shopping_queries_dataset_products.parquet')
df_sources = pd.read_csv("esci-data/shopping_queries_dataset/shopping_queries_dataset_sources.csv")
# df_dataset.head()

In [ ]:
df_examples.head()

In [ ]:
df_sources

In [ ]:
df_sources['source'].value_counts()

Merge examples with products


In [ ]:
df_examples_products = pd.merge(
    df_examples,
    df_products,
    how='left',
    left_on=['product_locale','product_id'],
    right_on=['product_locale', 'product_id']
)



Merge examples with query type


In [ ]:
# def isEnglish(s):
#     try:
#         s.encode(encoding='utf-8').decode('ascii')
#     except UnicodeDecodeError:
#         return False
#     else:
#         return True
    
    
# def is_english(s):
#     try:
#         return detect(s) == 'en'
#     except Exception as e:
#         print(( s,e) ) 
#         return False
#     return False

def is_english(text):
    # try:
    #     text = text.encode(encoding='utf-8').decode('ascii')
    # except UnicodeDecodeError as e:
    #     print(e)
    #     return False
    
    if re.search(r"[\u3000-\u303F]", text):
        return False
    
        

    allowed_characters = set(["$", "!", '"', "-", '#',",",".", "%", "'", '’', "’", ")" ,"(","”",'’','$', '”'])
    
    try:
        return all(c.isalnum() or c.isspace() or c in allowed_characters or c in string.punctuation for c in text)
    except:
        return False
    
def is_english(text):
    text = unicodedata.normalize('NFKC', text)
    
    lang = None
    try:
        lang = detect(text)
    except Exception as e:
        print (text, e)

    if lang in ['en','ja','es']:
        return lang
    
    text = unidecode(text)
    try:
        res = text.encode(encoding='utf-8').decode('ascii')
        return "en"
        
    except UnicodeDecodeError:
        pass
    
    
    

    
    return "unknown"


def contains_spanish_accents(text):
    spanish_accents = {'á', 'é', 'í', 'ó', 'ú', 'ñ'}
    return any(char in spanish_accents for char in text)


def contains_japanese(text):
    for char in text:
        if '\u3040' <= char <= '\u309F' or '\u30A0' <= char <= '\u30FF' or '\u4E00' <= char <= '\u9FAF':
            return True
    return False

# Example usage
def is_english(text):
    text = unicodedata.normalize('NFC', text)

    is_japanese = contains_japanese(text)

   
    if is_japanese:
        return "jp"
    
    is_spanish = contains_spanish_accents(text)
    
    if is_spanish:
        return "es"
    

    
    try:
        lang = detect(text)

        if lang =="en":
            return "en"

        #print(lang)

        
        allowed_characters = set(["$", "!", '"', "-", '#',",",".", "%", "'", '’', "’", ")" ,"(","”",'’','$', '”'])
    
        if all(c.isalnum() or c.isspace() or c in allowed_characters or c in string.punctuation for c in text):
            return "en"

        
        #text = unidecode(text)
        lang = detect(text)

        #print(text,lang)
        return lang
    except:
        return "unknown"

    
    
def clean_text(text):
    # Normalize Unicode characters
    text = unicodedata.normalize('NFKC', text)
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def detect_language(text):
    
    is_japanese = contains_japanese(text)
    if is_japanese:
        return "jp"
   
    is_spanish = contains_spanish_accents(text)
    
    if is_spanish:
        return "es"
    
    
    cleaned_text = clean_text(text)
    try:
        lang = detect(cleaned_text)
        if lang == 'en':
            return 'en'
        elif lang == 'ja':
            return 'ja'
        elif lang == 'es':
            return 'es'
        else:
            return 'unknown'
    except LangDetectException:
        return 'unknown'
    
    return 'unknown'

In [ ]:
chardet.detect("1” sink stopper and strainer".encode())

In [ ]:
is_english("1” sink stopper and strainer")

In [ ]:
is_english("!solid camiseta sin manga"), detect("!solid camiseta sin manga")

In [ ]:
is_english( 'alpro soja sin azúcar')

In [ ]:
is_english( '32ミリ コテ')

In [ ]:
text = "1” sink stopper and strainer"
text = unicodedata.normalize('NFKD', text)
text =unidecode(text)
print(text)
text.encode(encoding='utf-8').decode('ascii')

In [ ]:
?unidecode

In [ ]:
detect("War doesn't show who's right, just who's left.")

In [ ]:
detect_langs("Sink stopper")

In [ ]:
df_sources

In [ ]:
#df_examples_products_query['is_english'] =  df_examples_products_query['query'].apply(isEnglish)

In [ ]:
#detect("hi

In [ ]:
#df_query_lang = df_examples_products[df_examples_products ["product_locale"]=="us"]
df_query_lang = df_examples_products[['query','query_id']].drop_duplicates()



In [ ]:
df_query_lang

In [ ]:
df_query_lang['is_english'] = df_query_lang['query'].parallel_apply(is_english)


In [ ]:
df_query_lang['is_english'].value_counts()

In [ ]:
df_query_lang [ df_query_lang['is_english'].isin(['en']) ].head(500)

In [ ]:
df_query_lang [ ~df_query_lang['is_english'].isin(['en','ja','es']) ]

In [ ]:
"vitamina c".encode(encoding='utf-8').decode('ascii')

In [ ]:
df_examples_products_query = pd.merge(
    df_examples_products,
    df_query_lang,
    how='inner',
    on=['query_id','query']
)


In [ ]:
‘ . ´ × £ 

In [ ]:
list(df_query_lang [ ~df_query_lang['is_english'].isin(['en','jp','es']) ].head(10_000)['query'])

In [ ]:
is_english("$60 ps4 that’s not gonna be on amazon")

In [ ]:
is_english("mom life")

In [ ]:
detect("mom life")

In [ ]:
df_examples_products_query\
.pipe(lambda x: x[x.product_locale =="us"]) \
.replace({"esci_label": {'E':'Exact' ,'S':'Substitute', 'C':'Complement', 'I':'Irrelevant' } })\
.rename (columns = {'esci_label':'relevance_label','source':'query_type'} ) \
.drop(columns = ['example_id','query_id','product_locale','small_version','large_version','split'] )

In [ ]:
len(df_examples_products['product_id'].unique())

In [ ]:
df_mask = df_examples_products['product_locale'].isin(['us'])

len( df_examples_products[df_mask]['product_id'].unique()) 

In [ ]:
df_examples_products[df_mask]['product_id'].unique()

In [ ]:
df_dataset['source'].value_counts()

In [ ]:
df_product = pd.read_parquet("esci-data/shopping_queries_dataset/shopping_queries_dataset_products.parquet")
df_product.head()

In [ ]:
len(df_product)

In [ ]:
df_product['product_brand'].nunique()

In [ ]:
df_product['product_locale'].value_counts()

In [ ]:
df_product_en = df_product[df_product['product_locale']=="us"]

In [ ]:
df_product_en.iloc[500].to_dict()

In [ ]:
df_product_en.iloc[600].to_dict()

In [ ]:
df_examples = pd.read_parquet("esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet")
df_examples.head()

In [ ]:
df_examples['esci_label'].value_counts()

In [ ]:
len(df_examples)

In [ ]:
df_examples['query'].nunique()

In [ ]:
df_uniqeu_quereis = df_examples['query'].drop_duplicates()

In [ ]:
list ( df_uniqeu_quereis.head(100) )

In [ ]:
df_examples['product_id'].nunique()

In [ ]:
df_examples [df_examples['product_id']=="B000BQWKCY"]

In [ ]:
df_examples['query'].unique()

In [ ]:
df_examples [df_examples['query']=="$25 ceiling fan without lights"]